In [5]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [26]:
import numpy as np #Import relevant packages
import pandas as pd
import math
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import glob
import csv
import matplotlib.pyplot as plt
import datetime
import os

In [3]:
model = keras.models.load_model("/Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo/lstms/lstm1")

In [7]:
def unpack_data(filepath): #Returns a dictionary where each item is a df containing one run. One dict per generation.
    all_files = glob.glob(filepath + "/*.csv")
    name_list = []
    datadict = {}
    for f in all_files:
        name_list.append(f[77:-4])
    for i,n in enumerate(name_list):
        datadict[n] = pd.read_csv(all_files[i], header=0)
    for k, v in datadict.items():
        v.drop(columns=["Unnamed: 0", "volume", "spread", "10_MA", "50_MA"], inplace=True)
    return datadict

In [11]:
gen0 = unpack_data("/Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo//simulations/gen0_sims/data") #Load gen0 data

In [12]:
def format_data(datadict, window): #Turn dictionaries of data into features and samples lists
    x_data = []
    y_data = []
    for k, v in datadict.items():
        for i in range(len(v)):
            if i >= window:
                y_data.append(v.iloc[i]["trading_price"])
                xi = v.iloc[i-window:i].to_numpy()
                x_data.append(xi)
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    y_data = np.reshape(y_data, (y_data.shape[0], 1))
    return x_data, y_data

In [13]:
X, y = format_data(gen0, 20)
print(X.shape)
print(y.shape)

(31380, 20, 2)
(31380, 1)


In [14]:
def normalise_data(xarray, yarray): #Take the natural log and then normalise prices
    yarray_log = np.log(yarray)
    xarray_log = np.log(xarray)
    xsc = StandardScaler()
    instances, timesteps, features = xarray_log.shape
    xarray_log = np.reshape(xarray_log, (-1, features))
    xarray_norm = xsc.fit_transform(xarray_log)
    xarray_norm = np.reshape(xarray_norm, (instances, timesteps, features))
    ysc = StandardScaler().fit(yarray_log)
    yarray_norm = ysc.transform(yarray_log)
    return xarray_norm, yarray_norm, xsc, ysc

def split_data(xarray, yarray, trainratio): #Split the data into training and test sets
    train_len = int(len(xarray)*trainratio)
    test_len = len(xarray) - train_len
    x_train, y_train, x_test, y_test = xarray[:train_len], yarray[:train_len], xarray[train_len:], yarray[train_len:]
    return x_train, y_train, x_test, y_test

In [15]:
X_train, y_train, X_test, y_test = split_data(X, y, 0.8)
X_train, y_train, Xscale, yscale = normalise_data(X_train, y_train)
X_test, y_test, _, _ = normalise_data(X_test, y_test)

In [29]:
log_dir = "/Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [31]:
lstm1=keras.Sequential() #Set up the architecture of the model

lstm1.add(layers.LSTM(units=16, input_shape=(X_train.shape[1], X_train.shape[2])))
lstm1.add(layers.Dropout(0.2))

lstm1.add(layers.Dense(units=1))

lstm1.compile(optimizer='nadam', loss='mean_squared_error') #Compile and train the model

history1 = lstm1.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_split=0.2, shuffle=False, callbacks=[tensorboard_callback])

Train on 20083 samples, validate on 5021 samples
Epoch 1/10
20083/20083 [==============================] - 30s 1ms/sample - loss: 0.2348 - val_loss: 0.0077
Epoch 2/10
20083/20083 [==============================] - 22s 1ms/sample - loss: 0.0377 - val_loss: 0.0066
Epoch 3/10
20083/20083 [==============================] - 22s 1ms/sample - loss: 0.0280 - val_loss: 0.0176
Epoch 4/10
20083/20083 [==============================] - 23s 1ms/sample - loss: 0.0228 - val_loss: 0.0048
Epoch 5/10
20083/20083 [==============================] - 22s 1ms/sample - loss: 0.0218 - val_loss: 0.0021
Epoch 6/10
20083/20083 [==============================] - 22s 1ms/sample - loss: 0.0201 - val_loss: 0.0031
Epoch 7/10
20083/20083 [==============================] - 25s 1ms/sample - loss: 0.0196 - val_loss: 0.0021
Epoch 8/10
20083/20083 [==============================] - 25s 1ms/sample - loss: 0.0186 - val_loss: 0.0067
Epoch 9/10
20083/20083 [==============================] - 21s 1ms/sample - loss: 0.0183 - val_l

In [27]:
os.environ['TENSORBOARD_BINARY'] = '/Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo/to/envs/my_env/bin/tensorboard'

In [34]:
%tensorboard --log_dir logs/fit/

ERROR: Could not find '/Users/karangarg/Documents/Year 3
Modules/EC331/Code/rae_repo/to/envs/my_env/bin/tensorboard' (set by
the `TENSORBOARD_BINARY` environment variable). Please ensure that
your PATH contains an executable `tensorboard` program, or explicitly
specify the path to a TensorBoard binary by setting the
`TENSORBOARD_BINARY` environment variable.

In [35]:
rm -rf ./logs/

In [36]:
tensorboard --logdir="/Users/karangarg/Documents/Year 3 Modules/EC331/Code/rae_repo/logs/fit/"

ERROR: Could not find '/Users/karangarg/Documents/Year 3
Modules/EC331/Code/rae_repo/to/envs/my_env/bin/tensorboard' (set by
the `TENSORBOARD_BINARY` environment variable). Please ensure that
your PATH contains an executable `tensorboard` program, or explicitly
specify the path to a TensorBoard binary by setting the
`TENSORBOARD_BINARY` environment variable.